# Dependencies

In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


# Helper functions to draw poses and extract keypoints

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_lanmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw righ

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Basic landmark video testing function

In [8]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

# Video collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('C:\\Users\\Tomas\\Bakis\\MP_Data')
RAW_DATA_PATH = os.path.join('C:\\Users\\Tomas\\Bakis\\MP_Data\\raw') 

# Actions that we try to detect
actions = np.array(['suo', 'labas', 'aciu'])

# Thirty videos worth of data
no_sequences = 90

# Videos are going to be 30 frames in length
sequence_length = 60

In [20]:
import cv2
import mediapipe as mp

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

current_action = 'suo'

# Set mediapipe model
mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through sequences aka videos
    for sequence in range(no_sequences):
        # Video settings
        width, height = int(cap.get(3)), int(cap.get(4))
        fps = 29
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(os.path.join(RAW_DATA_PATH, current_action, str(sequence) + '.mp4'), fourcc, fps, (width, height))
                
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):
            # Read feed
            ret, frame = cap.read()
            # Write the frame to the output video
            out.write(frame)
            
             # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(current_action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)
                cv2.waitKey(1000)
            else: 
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(current_action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)

            # Perform your processing here

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    # Release the VideoWriter and VideoCapture objects
    out.release()
    cap.release()
    cv2.destroyAllWindows()

In [13]:
cap.release()
cv2.destroyAllWindows()

# Process data

In [21]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except ex:
            print(ex)
            pass

In [22]:
sequences, labels = [], []
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            cap = cv2.VideoCapture(os.path.join(RAW_DATA_PATH, action, str(sequence) + '.mp4'))
            frame_count = 0
    
            # Check if the video is successfully opened
            if not cap.isOpened():
                print("Error: Could not open video." + action + " " + str(sequence))
            
            while True:
                # Read a frame from the video
                ret, frame = cap.read()
                
                # Break the loop if the video has ended
                if not ret:
                    break

                image, results = mediapipe_detection(frame, holistic)
    
                # # Draw landmarks
                # draw_styled_landmarks(image, results)
                
                # # # Display the frame (optional)
                # cv2.imshow('Frame', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_count))
                print(npy_path)
                np.save(npy_path, keypoints)

                frame_count += 1
                        
                # Break the loop if 'q' is pressed
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            
            # Release the VideoCapture object
            cap.release()
            cv2.destroyAllWindows()

C:\Users\Tomas\Bakis\MP_Data\suo\0\0
C:\Users\Tomas\Bakis\MP_Data\suo\0\1
C:\Users\Tomas\Bakis\MP_Data\suo\0\2
C:\Users\Tomas\Bakis\MP_Data\suo\0\3
C:\Users\Tomas\Bakis\MP_Data\suo\0\4
C:\Users\Tomas\Bakis\MP_Data\suo\0\5
C:\Users\Tomas\Bakis\MP_Data\suo\0\6
C:\Users\Tomas\Bakis\MP_Data\suo\0\7
C:\Users\Tomas\Bakis\MP_Data\suo\0\8
C:\Users\Tomas\Bakis\MP_Data\suo\0\9
C:\Users\Tomas\Bakis\MP_Data\suo\0\10
C:\Users\Tomas\Bakis\MP_Data\suo\0\11
C:\Users\Tomas\Bakis\MP_Data\suo\0\12
C:\Users\Tomas\Bakis\MP_Data\suo\0\13
C:\Users\Tomas\Bakis\MP_Data\suo\0\14
C:\Users\Tomas\Bakis\MP_Data\suo\0\15
C:\Users\Tomas\Bakis\MP_Data\suo\0\16
C:\Users\Tomas\Bakis\MP_Data\suo\0\17
C:\Users\Tomas\Bakis\MP_Data\suo\0\18
C:\Users\Tomas\Bakis\MP_Data\suo\0\19
C:\Users\Tomas\Bakis\MP_Data\suo\0\20
C:\Users\Tomas\Bakis\MP_Data\suo\0\21
C:\Users\Tomas\Bakis\MP_Data\suo\0\22
C:\Users\Tomas\Bakis\MP_Data\suo\0\23
C:\Users\Tomas\Bakis\MP_Data\suo\0\24
C:\Users\Tomas\Bakis\MP_Data\suo\0\25
C:\Users\Tomas\Bakis\M

In [44]:
import os

folder_path = 'C:\\Users\\Tomas\\Bakis\\MP_data\\tomas2\\labas'  # Replace with the actual path to your folder

# Check if the path is a directory
if os.path.isdir(folder_path):
    # Iterate through folders and rename them
    for folder_name in os.listdir(folder_path):
        folder_path_old = os.path.join(folder_path, folder_name)
        
        # Check if the item is a directory
        if os.path.isdir(folder_path_old):
            # Extract the numeric part of the folder name
            try:
                old_number = int(folder_name)
                
                # Calculate the new folder name
                new_number = old_number + 60
                new_folder_name = str(new_number)
                
                # Construct the new folder path
                folder_path_new = os.path.join(folder_path, new_folder_name)
                
                # Rename the folder
                os.rename(folder_path_old, folder_path_new)
                
                print(f'Renamed: {folder_path_old} -> {folder_path_new}')
            
            except ValueError:
                print(f'Skipped: {folder_path_old} (not numeric)')
else:
    print(f'The provided path is not a directory: {folder_path}')

Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\0 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\60
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\1 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\61
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\10 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\70
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\11 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\71
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\12 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\72
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\13 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\73
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\14 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\74
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\15 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\75
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\16 -> C:\Users\Tomas\Bakis\MP_data\tomas2\labas\76
Renamed: C:\Users\Tomas\Bakis\MP_data\tomas2\labas\17 -> C:\Users\Tomas\Bakis\MP_dat

# Labeling dataset

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [74]:
label_map

{'suo': 0, 'labas': 1, 'aciu': 2}

In [75]:
import os

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)).replace("\\","\\"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [76]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [77]:
np.array(sequences).shape

(270, 60, 1662)

In [78]:
np.array(labels).shape

(270,)

In [79]:
x = np.array(sequences)

In [80]:
x.shape

(270, 60, 1662)

In [81]:
y = to_categorical(labels).astype(int)

In [82]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,

In [83]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [84]:
x_train.shape

(256, 60, 1662)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, Callback

In [11]:
log_dir = os.path.join('C:\\Users\\Tomas\\Bakis\\Logs')
tb_callback = TensorBoard(log_dir)

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [94]:
res = [.7, 0.2, 0.1]

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [96]:
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['categorical_accuracy'] >= 0.9:
            print(f"\nReached 0.9 accuracy, stopping training.")
            self.model.stop_training = True

In [97]:
custom_callback = CustomCallback()

In [98]:
model.fit(x_train, y_train, epochs=4000, callbacks=[tb_callback, custom_callback])

Epoch 1/4000
8/8 [==============================] - 3s 82ms/step - loss: 9.5050 - categorical_accuracy: 0.3750
Epoch 2/4000
8/8 [==============================] - 1s 75ms/step - loss: 94.3897 - categorical_accuracy: 0.2969
Epoch 3/4000
8/8 [==============================] - 1s 77ms/step - loss: 130.6720 - categorical_accuracy: 0.2734
Epoch 4/4000
8/8 [==============================] - 1s 74ms/step - loss: 4299.6704 - categorical_accuracy: 0.3164
Epoch 5/4000
8/8 [==============================] - 1s 76ms/step - loss: 6895.8594 - categorical_accuracy: 0.3633
Epoch 6/4000
8/8 [==============================] - 1s 76ms/step - loss: 164071.8125 - categorical_accuracy: 0.3008
Epoch 7/4000
8/8 [==============================] - 1s 75ms/step - loss: 152524.7188 - categorical_accuracy: 0.2852
Epoch 8/4000
8/8 [==============================] - 1s 76ms/step - loss: 55412.7383 - categorical_accuracy: 0.3281
Epoch 9/4000
8/8 [==============================] - 1s 75ms/step - loss: 258797.5156 - ca

KeyboardInterrupt: 

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 60, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

In [33]:
res = model.predict(x_test)

1/1 [==============================] - 0s 443ms/step


In [38]:
actions[np.argmax(res[1])]

'labas'

In [39]:
actions[np.argmax(y_test[1])]

'labas'

In [13]:
model.save('C:\\Users\\Tomas\\Bakis\\model-75acc-60frames.keras')

In [71]:
del model

In [15]:
model.load_weights('model-75acc-60frames.keras')

In [41]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [42]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 49ms/step


In [43]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 2],
        [2, 1]],

       [[4, 3],
        [1, 1]],

       [[4, 1],
        [3, 1]]], dtype=int64)

In [46]:
accuracy_score(ytrue, yhat)

0.3333333333333333

In [166]:
np.argmax(res)

0

In [50]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = []

            print(actions[np.argmax(res)])
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1) 
            cv2.putText(image, actions[np.argmax(res)], (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 30ms/step
labas
1/1 [==============================] - 0s 29ms/step
labas
1/1 [==============================] - 0s 30ms/step
suo
1/1 [==============================] - 0s 33ms/step
suo
1/1 [==============================] - 0s 32ms/step
labas
1/1 [==============================] - 0s 29ms/step
suo
1/1 [==============================] - 0s 31ms/step
suo
1/1 [==============================] - 0s 31ms/step
suo
1/1 [==============================] - 0s 31ms/step
suo
1/1 [==============================] - 0s 29ms/step
suo
1/1 [==============================] - 0s 31ms/step
suo
1/1 [==============================] - 0s 29ms/step
labas
1/1 [==============================] - 0s 35ms/step
labas


KeyboardInterrupt: 

In [44]:
res

array([0.00393879, 0.69763917, 0.29842204], dtype=float32)

In [45]:
actions

array(['suo', 'labas', 'aciu'], dtype='<U5')

In [23]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-60:]
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]

            if res[np.argmax(res)] > threshold:
                predictions.append(np.argmax(res))

                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                     cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
                     cv2.putText(image, actions[np.argmax(res)], (3,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [21]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    sequence = []
    
    # cap = cv2.VideoCapture('C:\\Users\\Tomas\\Bakis\\MP_Data\\raw\\tomas\\suo\\1.mp4')
    # cap = cv2.VideoCapture('C:\\Users\\Tomas\\Bakis\\MP_Data\\raw\\tomas\\labas\\4.mp4')
    cap = cv2.VideoCapture('C:\\Users\\Tomas\\Bakis\\MP_Data\\raw\\tomas\\aciu\\7.mp4')
    
    frame_count = 0
    
    # Check if the video is successfully opened
    if not cap.isOpened():
        print("Error: Could not open video." + action + " " + str(sequence))
    
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        
        # Break the loop if the video has ended
        if not ret:
            break
    
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        
        cv2.imshow('Frame', image)
    
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
                
        # Break the loop if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    res = model.predict(np.expand_dims(sequence, axis=0))[0]
    print(actions[np.argmax(res)])
    
    # Release the VideoCapture object
    cap.release()
    cv2.destroyAllWindows()   

1/1 [==============================] - 0s 40ms/step
suo


In [41]:
import os
import cv2

# Set the path to your directory containing video files
directory_path = 'C:\\Users\\Tomas\\Bakis\\MP_Data\\raw\\tomas2\\labas'

# Get a list of all files in the directory
all_files = os.listdir(directory_path)

# Filter for video files (you can add more video file extensions if needed)
video_files = [file for file in all_files if file.lower().endswith(('.mp4', '.avi', '.mkv'))]

# Iterate through each video file
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_file in video_files:
        # Construct the full path to the video file
        video_path = os.path.join(directory_path, video_file)
    
        # Open the video file using OpenCV
        cap = cv2.VideoCapture(video_path)
        sequence = []
        
        while True:
            # Read a frame from the video
            ret, frame = cap.read()
            
            # Break the loop if the video has ended
            if not ret:
                break
        
            image, results = mediapipe_detection(frame, holistic)
        
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
                    
            # Break the loop if 'q' is pressed
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(actions[np.argmax(res)])

# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()  

1/1 [==============================] - 0s 30ms/step
aciu
1/1 [==============================] - 0s 29ms/step
aciu
1/1 [==============================] - 0s 30ms/step
labas
1/1 [==============================] - 0s 31ms/step
aciu
1/1 [==============================] - 0s 28ms/step
suo
1/1 [==============================] - 0s 31ms/step
labas
1/1 [==============================] - 0s 29ms/step
labas
1/1 [==============================] - 0s 30ms/step
labas
1/1 [==============================] - 0s 29ms/step
labas
1/1 [==============================] - 0s 29ms/step
labas
1/1 [==============================] - 0s 28ms/step
labas
1/1 [==============================] - 0s 28ms/step
labas
1/1 [==============================] - 0s 28ms/step
labas
1/1 [==============================] - 0s 32ms/step
labas
1/1 [==============================] - 0s 34ms/step
labas
1/1 [==============================] - 0s 28ms/step
labas
1/1 [==============================] - 0s 30ms/step
labas
1/1 [==============

In [52]:
import cv2
import mediapipe as mp

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

current_action = 'suo'

# Set mediapipe model
mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Video settings
        width, height = int(cap.get(3)), int(cap.get(4))
        fps = 29
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(os.path.join(RAW_DATA_PATH, 'test', 'tomas-labas.mp4'), fourcc, fps, (width, height))
        print(os.path.join(RAW_DATA_PATH, 'test', 'tomas-labas.mp4'))
                
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):
            # Read feed
            ret, frame = cap.read()
            # Write the frame to the output video
            out.write(frame)
            
             # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(current_action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)
                cv2.waitKey(1000)
            else: 
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(current_action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)

            # Perform your processing here

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

# Release the VideoWriter and VideoCapture objects
out.release()
cap.release()
cv2.destroyAllWindows()

C:\Users\Tomas\Bakis\MP_Data\raw\test\tomas-labas.mp4


In [51]:
cap.release()
cv2.destroyAllWindows()